In [178]:
from midiutil import MIDIFile
import rebound
import numpy as np
import matplotlib.pyplot as plt
from subprocess import call

In [192]:
sim = rebound.Simulation.from_file("trappist.bin")
ps = sim.particles

Norbits = 10 # length of clip in number of orbits of the of outer planet

# 12 notes between octaves, freq = f0*2**(n/12). 
# n = 12 log_2(freq/f0)
# star, then planets from inside out
freqs = [744.96, 1624.73500794, 1013.5641465, 606.14619962, 402.42797069, 266.61557887, 198.71039594, 130.81]
basefreq = freqs[-1]

midinotes = []
for freq in freqs:
    midinote = 48+12*np.log(freq/basefreq)/np.log(2)
    midinotes.append(int(np.round(midinote)))

bodies = list(range(sim.N))
notes = dict(zip(bodies, midinotes))

channel  = 0
time     = 0   # In beats
duration = 1   # In beats
tempo    = 60  # In BPM. One beat = one outer planet orbit.
volume   = 100 # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1, adjust_origin=True) # One track, defaults to format 1 (tempo track
                     # automatically created)
MyMIDI.addTempo(0, time, tempo)

sim.t = 0
sim.dt = ps[1].P/100
tmax = Norbits*ps[-1].P

yprev = np.zeros(sim.N)
while sim.t < tmax:
    sim.step()
    for j in range(1,sim.N):
        if yprev[j] < 0 and ps[j].y > 0:
            print(sim.t, beat, j)
            MyMIDI.addNote(0, channel, notes[j], sim.t/ps[-1].P, duration, volume)
        yprev[j] = ps[j].y
    beat += 1
   
with open("midi.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

/Users/dtamayo/Documents/workspace/rebound/rebound/simulation.py:374: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/Users/dtamayo/Documents/workspace/rebound/rebound/simulation.py:374: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


1.881110668574806 241911 1
1.9986800853607314 241913 7
3.879790753935551 241945 2
7.7595815078711174 242011 1
9.934615718410752 242048 6
13.344128805202613 242106 2
13.638052347167429 242111 1
15.16645476538447 242137 3
19.51652318646374 242211 1
22.74968214807671 242266 2
23.63145277397116 242281 4
25.394994025760052 242311 1
30.568048364340807 242399 5
30.861971906305623 242404 3
31.273464865056365 242411 1
32.15523549095081 242426 2
37.151935704352674 242511 1
41.56078883382491 242586 2
43.030406543648986 242611 1
46.557489047226774 242671 3
47.145336131156405 242681 4
48.9088773829453 242711 1
50.96634217669901 242746 2
54.78734822224161 242811 1
57.13873655796014 242851 6
60.37189551957311 242906 2
60.665819061537924 242911 1
62.25300618814793 242938 3
65.95644281690413 243001 5
66.54428990083362 243011 1
69.77744886244581 243066 2
70.71800419673299 243082 4
72.42276074012851 243111 1
73.06939253245095 243122 7
77.94852332906571 243205 3
78.3012315794234 243211 1
79.18300220531764

In [193]:
call("open midi.mid", shell=True)

0

Period ratios

In [94]:
for i in range(1,7):
    print(ps[i+1].P/ps[i].P)

1.6009208063239937
1.6679113018571639
1.5009741278685107
1.501939182994456
1.335390113399473
1.5037062845692097
